In [22]:
import pandas as pd

In [23]:
test_df = pd.read_csv('/Users/alexis/Cranfield/AI/assignment/data/test.csv')
test_df.head()

,image_id,class_name,rad_id,x_min,y_min,x_max,y_max,class_id
0,27b822c5d3b354f096dfb788fd3fa636,Nodule/Mass,R10,277.0,602.0,1177.0,2417.0,8
1,d2ca714f8f06979acfbc0992d65fb211,No finding,R3,NaN,NaN,NaN,NaN,14
2,19367e88f4634c2652d3eed0a2f1f8b2,No finding,R15,NaN,NaN,NaN,NaN,14
3,852ec6a0bcb9608ffee9b3fda0867f91,Aortic enlargement,R8,1669.0,896.0,1972.0,1326.0,0
4,aa9c3097e6d2df7216be1a1642f683c9,No finding,R5,NaN,NaN,NaN,NaN,14


In [24]:
test_pred = pd.read_csv('/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/test_predictions.csv')
test_pred.head()

,image_id,PredictionString
0,27b822c5d3b354f096dfb788fd3fa636,2 0.9649 85.22254943847656 134.89244079589844 ...
1,d2ca714f8f06979acfbc0992d65fb211,14 1.0 0 0 1 1
2,19367e88f4634c2652d3eed0a2f1f8b2,4 0.5823 48.96579360961914 39.28007125854492 1...
3,852ec6a0bcb9608ffee9b3fda0867f91,8 0.5704 151.9897918701172 48.62690734863281 2...
4,aa9c3097e6d2df7216be1a1642f683c9,14 1.0 0 0 1 1


In [25]:
metadata_df = pd.read_csv('/Users/alexis/Cranfield/AI/assignment/dicom_metadata.csv')
metadata_df.head()

,File Name,Transfer Syntax UID,SOP Class UID,SOP Instance UID,Version,Photometric Interpretation,Pixel Representation,High Bit,Samples per Pixel,Bits Allocated,...,Lossy Image Compression,Lossy Image Compression Ratio,Patient's Sex,Patient's Age,Rows,Columns,Pixel Spacing,Bits Stored,Window Center,Window Width
0,f7f7736401b22c94a9d2b09425adeab9.dicom,1.2.840.10008.1.2.4.90,1.2.840.10008.5.1.4.1.1.1.1,f7f7736401b22c94a9d2b09425adeab9,INF_3.9,MONOCHROME2,0,11,1,16,...,0.0,NaN,F,037Y,3028,2517,"[0.139, 0.139]",12,2048.0,4096.0
1,93ae57ca1df1d19c6d64155e74e14d23.dicom,1.2.840.10008.1.2,1.2.840.10008.5.1.4.1.1.1.1,93ae57ca1df1d19c6d64155e74e14d23,NaN,MONOCHROME2,0,13,1,16,...,NaN,NaN,O,Y,3072,3072,"[0.139000, 0.139000]",14,10287.0,5292.0
2,f1a45afaee0efd07fef17057f3942464.dicom,1.2.840.10008.1.2.4.90,1.2.840.10008.5.1.4.1.1.1,f1a45afaee0efd07fef17057f3942464,OFFIS_DCMTK_360,MONOCHROME1,0,11,1,16,...,0.0,NaN,M,NaN,2880,2304,"[0.15, 0.15]",12,2047.0,4095.0
3,559fdfb3b9db3eb206b9521824f716e3.dicom,1.2.840.10008.1.2,1.2.840.10008.5.1.4.1.1.1,559fdfb3b9db3eb206b9521824f716e3,OFFIS_DCMTK_360,MONOCHROME2,0,13,1,16,...,NaN,NaN,M,038Y,3072,2540,"[0.140, 0.140]",14,10075.0,10242.0
4,371d6f540ac601affe81e547b05c551a.dicom,1.2.840.10008.1.2,1.2.840.10008.5.1.4.1.1.1,371d6f540ac601affe81e547b05c551a,OFFIS_DCMTK_360,MONOCHROME2,0,13,1,16,...,NaN,NaN,M,060Y,3072,3072,"[0.140, 0.140]",14,8755.0,10228.0


In [26]:
image_sizes_df = metadata_df[['File Name', 'Rows', 'Columns']]
image_sizes_df.head()

,File Name,Rows,Columns
0,f7f7736401b22c94a9d2b09425adeab9.dicom,3028,2517
1,93ae57ca1df1d19c6d64155e74e14d23.dicom,3072,3072
2,f1a45afaee0efd07fef17057f3942464.dicom,2880,2304
3,559fdfb3b9db3eb206b9521824f716e3.dicom,3072,2540
4,371d6f540ac601affe81e547b05c551a.dicom,3072,3072


In [ ]:
import logging
import h5py

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

def get_pixel_array_from_hdf5(hdf5_path, filename):
    """
    Retrieves the pixel array for a given filename from an HDF5 file.

    Parameters:
    - hdf5_path: Path to the HDF5 file.
    - filename: The original file name of the DICOM image.

    Returns:
    - A numpy array of the pixel data if found; None otherwise.
    """
    with h5py.File(hdf5_path, "r") as hdf5_file:
        # Attempt to access the dataset directly by filename.
        # Adjust this part if a more complex naming convention is used.
        unique_filename = None
        for key in hdf5_file.keys():
            if filename in key:
                unique_filename = key
                break
                
        if unique_filename is not None:
            pixel_array = hdf5_file[unique_filename][:]
            return pixel_array
        else:
            logging.error(f"File {filename} not found in HDF5.")
            return None

In [ ]:
import matplotlib.pyplot as plt

filename = "f7f7736401b22c94a9d2b09425adeab9"  # This should be the base name of the file you're looking for.
pixel_array = get_pixel_array_from_hdf5("/Volumes/ALEXIS/ai_project_cranfield/dicom_images_final.hdf5", filename+".dicom")
plt.imshow(pixel_array, cmap="gray")

# print the size of the image
original_size = pixel_array.shape[:2]  # (hauteur, largeur)
print(original_size)

In [27]:
test_prediction_reformatted = pd.read_csv("/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/test_predictions_reformatted.csv")
test_prediction_reformatted

,image_id,class_id,score,x_min,y_min,x_max,y_max
0,27b822c5d3b354f096dfb788fd3fa636,2,0.9649,85.222549,134.892441,164.499115,169.483978
1,27b822c5d3b354f096dfb788fd3fa636,7,0.8045,69.390923,117.481094,83.706902,133.776917
2,27b822c5d3b354f096dfb788fd3fa636,7,0.8036,51.284595,115.132790,59.561249,121.462784
3,27b822c5d3b354f096dfb788fd3fa636,7,0.7901,34.616817,162.675385,41.357605,170.029221
4,27b822c5d3b354f096dfb788fd3fa636,4,0.7735,132.583740,49.773079,192.411087,171.798676
...,...,...,...,...,...,...,...
13864,eb6c714df22142229464c6b83e47d7d6,7,0.8023,156.411926,98.928635,179.730545,116.733383
13865,fcb84a0cb71696938093ddb14dba07e3,8,0.8723,192.409119,27.605824,223.156555,179.792114
13866,fcb84a0cb71696938093ddb14dba07e3,8,0.7873,4.619601,23.875128,44.756027,169.450165
13867,fcb84a0cb71696938093ddb14dba07e3,8,0.7617,178.138718,34.956375,223.806686,121.604752


In [28]:
image_sizes_df = image_sizes_df.rename(columns={"File Name": "image_id"})
# Drop .dicom from the image_id
image_sizes_df["image_id"] = image_sizes_df["image_id"].str.replace(".dicom", "")
image_sizes_df

,image_id,Rows,Columns
0,f7f7736401b22c94a9d2b09425adeab9,3028,2517
1,93ae57ca1df1d19c6d64155e74e14d23,3072,3072
2,f1a45afaee0efd07fef17057f3942464,2880,2304
3,559fdfb3b9db3eb206b9521824f716e3,3072,2540
4,371d6f540ac601affe81e547b05c551a,3072,3072
...,...,...,...
14995,0c5ff01c7bfb4362fcd98f36e555b08c,2880,2304
14996,3286115baacb6ed3affc5c46c9205b7a,2880,2304
14997,d44766bf0eb902fbc1848f418417c45f,2208,2104
14998,cd7b83d24ee9c092a0f32cb020da6dac,2926,2847


In [29]:
test_prediction_reformatted = test_prediction_reformatted.merge(image_sizes_df, on="image_id", how="left")
test_prediction_reformatted

,image_id,class_id,score,x_min,y_min,x_max,y_max,Rows,Columns
0,27b822c5d3b354f096dfb788fd3fa636,2,0.9649,85.222549,134.892441,164.499115,169.483978,2968,2580
1,27b822c5d3b354f096dfb788fd3fa636,7,0.8045,69.390923,117.481094,83.706902,133.776917,2968,2580
2,27b822c5d3b354f096dfb788fd3fa636,7,0.8036,51.284595,115.132790,59.561249,121.462784,2968,2580
3,27b822c5d3b354f096dfb788fd3fa636,7,0.7901,34.616817,162.675385,41.357605,170.029221,2968,2580
4,27b822c5d3b354f096dfb788fd3fa636,4,0.7735,132.583740,49.773079,192.411087,171.798676,2968,2580
...,...,...,...,...,...,...,...,...,...
13864,eb6c714df22142229464c6b83e47d7d6,7,0.8023,156.411926,98.928635,179.730545,116.733383,3073,2532
13865,fcb84a0cb71696938093ddb14dba07e3,8,0.8723,192.409119,27.605824,223.156555,179.792114,2500,2048
13866,fcb84a0cb71696938093ddb14dba07e3,8,0.7873,4.619601,23.875128,44.756027,169.450165,2500,2048
13867,fcb84a0cb71696938093ddb14dba07e3,8,0.7617,178.138718,34.956375,223.806686,121.604752,2500,2048


In [31]:
test_prediction_reformatted = test_prediction_reformatted.rename(columns={"Rows": "image_height", "Columns": "image_width"})

In [32]:
def resizeBox(x, x_target, x_original):
    return x * x_target / x_original

target_size = (800, 800)

test_prediction_reformatted["x_min"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.x_min, target_size[0], x.image_width), axis=1)
test_prediction_reformatted["x_max"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.x_max, target_size[0], x.image_width), axis=1)
test_prediction_reformatted["y_min"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.y_min, target_size[1], x.image_height), axis=1)
test_prediction_reformatted["y_max"] = test_prediction_reformatted.apply(lambda x: resizeBox(x.y_max, target_size[1], x.image_height), axis=1)

In [33]:
test_prediction_reformatted

,image_id,class_id,score,x_min,y_min,x_max,y_max,image_height,image_width
0,27b822c5d3b354f096dfb788fd3fa636,2,0.9649,26.425597,36.359148,51.007478,45.683013,2968,2580
1,27b822c5d3b354f096dfb788fd3fa636,7,0.8045,21.516565,31.666063,25.955628,36.058468,2968,2580
2,27b822c5d3b354f096dfb788fd3fa636,7,0.8036,15.902200,31.033097,18.468604,32.739295,2968,2580
3,27b822c5d3b354f096dfb788fd3fa636,7,0.7901,10.733897,43.847813,12.824064,45.829979,2968,2580
4,27b822c5d3b354f096dfb788fd3fa636,4,0.7735,41.111237,13.415924,59.662353,46.306921,2968,2580
...,...,...,...,...,...,...,...,...,...
13864,eb6c714df22142229464c6b83e47d7d6,7,0.8023,49.419250,25.754282,56.786902,30.389426,3073,2532
13865,fcb84a0cb71696938093ddb14dba07e3,8,0.8723,75.159812,8.833864,87.170529,57.533477,2500,2048
13866,fcb84a0cb71696938093ddb14dba07e3,8,0.7873,1.804532,7.640041,17.482823,54.224053,2500,2048
13867,fcb84a0cb71696938093ddb14dba07e3,8,0.7617,69.585437,11.186040,87.424487,38.913521,2500,2048


In [34]:
test_df

,image_id,class_name,rad_id,x_min,y_min,x_max,y_max,class_id
0,27b822c5d3b354f096dfb788fd3fa636,Nodule/Mass,R10,277.0,602.0,1177.0,2417.0,8
1,d2ca714f8f06979acfbc0992d65fb211,No finding,R3,NaN,NaN,NaN,NaN,14
2,19367e88f4634c2652d3eed0a2f1f8b2,No finding,R15,NaN,NaN,NaN,NaN,14
3,852ec6a0bcb9608ffee9b3fda0867f91,Aortic enlargement,R8,1669.0,896.0,1972.0,1326.0,0
4,aa9c3097e6d2df7216be1a1642f683c9,No finding,R5,NaN,NaN,NaN,NaN,14
...,...,...,...,...,...,...,...,...
10183,54e6184c63c75a9695d7effc17969ad0,Cardiomegaly,R8,731.0,1431.0,1697.0,1842.0,3
10184,7237fe007c5cab239011e89137eee3a7,Other lesion,R17,667.0,752.0,1282.0,1777.0,9
10185,3b2087f26637d64244ec11da41770aad,Aortic enlargement,R9,1604.0,705.0,1850.0,931.0,0
10186,eb6c714df22142229464c6b83e47d7d6,Lung Opacity,R10,494.0,1293.0,866.0,1632.0,7
